# TensorFlow Lite Model Conversion for On-Device Training

In [ ]:
import sys

sys.path.append("../src")

from src.models import OnDeviceTrainingModel
import tensorflow as tf
import numpy as np

In [ ]:
# Create on-device training model using our class
model = OnDeviceTrainingModel(num_classes=10, img_size=160, learning_rate=1.4e-5)

# Load pre-trained weights if available
# model.model.load_weights("/path/to/your/weights.h5")

print("On-device training model created successfully!")
print(f"Model input shape: {model.model.input_shape}")
print(f"Model output shape: {model.model.output_shape}")

In [ ]:
# Convert to TensorFlow Lite using our built-in method
saved_model_dir = "on_device_training_model"
tflite_model = model.convert_to_tflite(saved_model_dir)

# Save the TFLite model
tflite_filename = 'on_device_training_model.tflite'
with open(tflite_filename, 'wb') as f:
    f.write(tflite_model)

print(f"Model successfully converted to TensorFlow Lite!")
print(f"Saved as: {tflite_filename}")
print(f"Model size: {len(tflite_model) / 1024 / 1024:.2f} MB")

In [ ]:
# Test the TFLite model
interpreter = tf.lite.Interpreter(model_path=tflite_filename)

# Get model signatures
signatures = interpreter.get_signature_list()
print('Available signatures:', list(signatures.keys()))

# Test inference signature
interpreter.allocate_tensors()
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()

print(f"\nInference input shape: {input_details[0]['shape']}")
print(f"Inference output shape: {output_details[0]['shape']}")

# Create dummy input for testing
test_input = np.random.random((1, 160, 160, 3)).astype(np.float32)
interpreter.set_tensor(input_details[0]['index'], test_input)
interpreter.invoke()
output = interpreter.get_tensor(output_details[0]['index'])

print(f"Test inference successful! Output shape: {output.shape}")
print("Model is ready for on-device deployment!")

In [ ]:

pip install tensorflow==2.15


In [ ]:
import numpy as np
import tensorflow_datasets as tfds
#import tensorflow as tf  # For tf.data
import matplotlib.pyplot as plt
import keras
from tensorflow import keras
from keras import layers
from keras.applications import EfficientNetB0
from keras.callbacks import EarlyStopping
from sklearn.metrics import confusion_matrix
from sklearn.metrics import f1_score, precision_score, recall_score
import seaborn as sns
import pickle

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf
print("TensorFlow version:", tf.__version__)


TensorFlow version: 2.15.0


In [ ]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


Load model trained on server

In [ ]:
IMAGE_SHAPE = (160, 160, 3)
IMG_SIZE = 160
def build_model(num_classes):

    base_model = tf.keras.applications.MobileNetV2(input_shape=IMAGE_SHAPE, include_top=False, alpha=1.0)

    base_model.trainable = False

    # Rebuild top
    x = base_model.output
    x = tf.keras.layers.GlobalAveragePooling2D(name="avg_pool")(x)

    top_dropout_rate = 0.1
    x = tf.keras.layers.Dropout(top_dropout_rate, name="top_dropout1")(x)

    outputs = tf.keras.layers.Dense(num_classes, activation="softmax", name="pred")(x)

    model = tf.keras.Model(base_model.input, outputs)

    return model

In [ ]:
class Model(tf.Module):

  def __init__(self):

    self.model = build_model(num_classes=10)
    self.model.trainable = True
    for layer in self.model.layers:
      if isinstance(layer, layers.BatchNormalization):
          layer.trainable = False
    self.opt = tf.keras.optimizers.Adam(learning_rate=1.4e-5, epsilon=0.002, amsgrad=True, weight_decay=1e-5)
    self.Loss = tf.keras.losses.CategoricalCrossentropy(label_smoothing=0.1)
    self.model.compile(optimizer=self.opt, loss=self.Loss, metrics=["accuracy"])

  # The `train` function takes a batch of input images and labels.
  @tf.function(input_signature=[
      tf.TensorSpec([5, IMG_SIZE, IMG_SIZE, 3], tf.float32),
      tf.TensorSpec([5, 10], tf.float32),
  ])
  def train(self, x, y):
    with tf.GradientTape() as tape:
      prediction = self.model(x,training=True)  # next approach remove the training=true
      loss = self.model.loss(y, prediction)
    gradients = tape.gradient(loss, self.model.trainable_variables)
    self.model.optimizer.apply_gradients(
        zip(gradients, self.model.trainable_variables))

    result = {"loss": loss}
    return result

  @tf.function(input_signature=[
      tf.TensorSpec([None, IMG_SIZE, IMG_SIZE, 3], tf.float32),
  ])
  def infer(self, x):
    probabilities = self.model(x)
    return {"output": probabilities}

  @tf.function(input_signature=[tf.TensorSpec(shape=[], dtype=tf.string)])
  def save(self, checkpoint_path):
    tensor_names = [weight.name for weight in self.model.weights]
    tensors_to_save = [weight.read_value() for weight in self.model.weights]
    tf.raw_ops.Save(
        filename=checkpoint_path, tensor_names=tensor_names,
        data=tensors_to_save, name='save')
    return {
        "checkpoint_path": checkpoint_path
    }

  @tf.function(input_signature=[tf.TensorSpec(shape=[], dtype=tf.string)])
  def restore(self, checkpoint_path):
    restored_tensors = {}
    for var in self.model.weights:
      restored = tf.raw_ops.Restore(
          file_pattern=checkpoint_path, tensor_name=var.name, dt=var.dtype,
          name='restore')
      var.assign(restored)
      restored_tensors[var.name] = restored
    return restored_tensors


In [ ]:
m = Model()
m.model.load_weights("/content/drive/MyDrive/Diploma thesis/mnv2_cifar8_160.weights.h5")  # I CHANGED FOR 8-2 SPLIT !!!!



SAVED_MODEL_DIR = "the_saved_model"
tf.saved_model.save(
    m,
    SAVED_MODEL_DIR,
    signatures={
        'train':
            m.train.get_concrete_function(),
        'infer':
            m.infer.get_concrete_function(),
        'save':
            m.save.get_concrete_function(),
        'restore':
            m.restore.get_concrete_function(),
    })

# Convert the model
converter = tf.lite.TFLiteConverter.from_saved_model(SAVED_MODEL_DIR)
converter.target_spec.supported_ops = [
    tf.lite.OpsSet.TFLITE_BUILTINS,  # enable TensorFlow Lite ops.
    tf.lite.OpsSet.SELECT_TF_OPS  # enable TensorFlow ops.
]

converter.experimental_enable_resource_variables = True
tflite_model = converter.convert()

# Save the model
with open('lr_014_model.tflite', 'wb') as f:
  f.write(tflite_model)


9406464/9406464 [==============================] - 0s 0us/step


In [ ]:
import tensorflow as tf

# Load the TensorFlow Lite model
interpreter = tf.lite.Interpreter(model_path='/content/drive/MyDrive/model.tflite')

signatures = interpreter.get_signature_list()
print('Signature:', signatures)




Signature: {'infer': {'inputs': ['x'], 'outputs': ['output']}, 'restore': {'inputs': ['checkpoint_path'], 'outputs': ['Conv1/kernel:0', 'Conv_1/kernel:0', 'Conv_1_bn/beta:0', 'Conv_1_bn/gamma:0', 'Conv_1_bn/moving_mean:0', 'Conv_1_bn/moving_variance:0', 'block_10_depthwise/depthwise_kernel:0', 'block_10_depthwise_BN/beta:0', 'block_10_depthwise_BN/gamma:0', 'block_10_depthwise_BN/moving_mean:0', 'block_10_depthwise_BN/moving_variance:0', 'block_10_expand/kernel:0', 'block_10_expand_BN/beta:0', 'block_10_expand_BN/gamma:0', 'block_10_expand_BN/moving_mean:0', 'block_10_expand_BN/moving_variance:0', 'block_10_project/kernel:0', 'block_10_project_BN/beta:0', 'block_10_project_BN/gamma:0', 'block_10_project_BN/moving_mean:0', 'block_10_project_BN/moving_variance:0', 'block_11_depthwise/depthwise_kernel:0', 'block_11_depthwise_BN/beta:0', 'block_11_depthwise_BN/gamma:0', 'block_11_depthwise_BN/moving_mean:0', 'block_11_depthwise_BN/moving_variance:0', 'block_11_expand/kernel:0', 'block_11_e

In [ ]:
interpreter.allocate_tensors()

# Get input details
input_details = interpreter.get_input_details()
input_index = input_details[0]['index']

# Define new tensor size
new_size = [5, 160, 160, 3]  # Example: Changing batch size to 5

# Resize the input tensor
interpreter.resize_tensor_input(input_index, new_size, strict=False)

# Allocate tensors again after resizing
interpreter.allocate_tensors()

In [ ]:
# Get input tensor details
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()

# Print input tensor details
for i, input_tensor in enumerate(input_details):
    print(f"Input Tensor {i}:")
    print(f"Name: {input_tensor['name']}")
    print(f"Shape: {input_tensor['shape']}")
    print(f"Data Type: {input_tensor['dtype']}")

# Print output tensor details
for i, output_tensor in enumerate(output_details):
    print(f"Output Tensor {i}:")
    print(f"Name: {output_tensor['name']}")
    print(f"Shape: {output_tensor['shape']}")
    print(f"Data Type: {output_tensor['dtype']}")

Input Tensor 0:
Name: infer_x:0
Shape: [  5 160 160   3]
Data Type: <class 'numpy.float32'>
Output Tensor 0:
Name: StatefulPartitionedCall:0
Shape: [ 5 10]
Data Type: <class 'numpy.float32'>
